In [2]:
import numpy as np
import matplotlib as plt
import random

import keras
(train_inputs, train_labels), (test_inputs, test_labels) = keras.datasets.mnist.load_data()

train_inputs = train_inputs.astype("float32") / 255.0
test_inputs = test_inputs.astype("float32") / 255.0

test_inputs = test_inputs.astype("int32")
test_inputs = test_inputs.astype("int32")

training_data = [np.array(x.flatten()) for x in train_inputs]
test_data = [np.array(x.flatten()) for x in test_inputs]

2024-07-08 16:20:11.414625: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [3]:
print(training_data[1])

[0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         0.
 0.         0.         0.         0.         0.         

In [2]:
class NeuralNet:
    def __init__(self, layers):
        # layers: list with how many neurons in each layer
        self.layers = layers
        self.weights = np.random.rand(10, 784) * 0.001
        # assuming a bias of 0

    def get_weights(self):
        return self.weights

    def set_weights(self, weights):
        self.weights = weights

    def softmax(self, x):
        # x: a 1d np.array
        # returns the softmax value
        x = x - np.max(x)
        exp_x = np.exp(x)
        softmax_x = exp_x / np.sum(exp_x)
        return softmax_x

    def randomize_array(self, arr):
        # returns a slightly randomized, noisy np.array of the input np.array
        noise = np.random.normal(0, 0.5, arr.shape)
        randomized_arr = arr + noise
        return randomized_arr

    def vectorized_result(self, x):
        # returns a np.array with 0, except for a 1 in the x index
        return_vec = np.zeros(10)
        return_vec[int(x)] = 1
        return return_vec

    def forward_pass(self, input, weights):
        return self.softmax(np.dot(weights, input))

    def identify(self, input):
        return np.argmax(input)

    def cross_entropy_loss(self, actual, prediction):
        epsilon = 1e-15  # to prevent log(0)
        y_pred = np.clip(prediction, epsilon, 1 - epsilon)
        return -np.mean(actual * np.log(y_pred) + (1 - actual) * np.log(1 - y_pred))

    def train(self, training_data, labels):
        pairs = list(zip(training_data, labels))
        for i in range(100): # set number of epochs to 100
            random.shuffle(pairs)
            n = len(pairs)
            mini_batches = [pairs[k:k+2500] for k in range(0, n, 2500)] # set batch size to 100 to train
            weights_list = []
            loss_list = []
            for data, label in mini_batches[0]:
                actual_vector = self.vectorized_result(label)
                # bias the weight matrices 3 times and multiply by error and find the mean:
                new_weight_list = []
                new_loss_list = []

                # add the current weights
                pred = self.softmax(self.forward_pass(data, self.weights))
                this_loss = self.cross_entropy_loss(pred, actual_vector)
                new_loss_list.append(this_loss)
                new_weight_list.append(self.weights)
                # create slight variations and evolutions
                for j in range(5):
                    curr_weight = self.randomize_array(self.weights)
                    new_weight_list.append(curr_weight)
                    prediction = self.softmax(self.forward_pass(data, curr_weight))
                    loss = self.cross_entropy_loss(prediction, actual_vector)
                    new_loss_list.append(loss)

                # pick best evolution or curent weight, that is the one with the least amount of loss
                new_weight_array = np.array(new_weight_list)
                new_loss_array = np.array(new_loss_list)
                index = np.argmin(new_loss_array)
                weights_list.append(new_weight_array[index] - self.weights) # updated to only add the noise vector
                loss_list.append(new_loss_array[index])

            weight_array = np.array(weights_list)
            loss_array = np.array(loss_list)
            normalized = ((loss_array - loss_array.min()) / (loss_array.max() - loss_array.min())) - 0.5
            gradient = np.zeros(weight_array[0].shape)
            for j in range(len(weights_list)):
                gradient += weight_array[j] * normalized[j]
            gradient /= len(weights_list)
            self.weights -= 0.01 * gradient # learning rate = 0.01

    def test_accuracy(self):
        total_num_trials = 0
        correct = 0
        pairs = zip(test_data, test_labels)
        for input, label in pairs:
            x = self.forward_pass(input, self.weights)
            id = self.identify(x)
            if (id == label):
                correct += 1
            total_num_trials += 1
        return (correct / total_num_trials)
        # print(self.weights)

In [3]:
for i in range(6):
    net = NeuralNet([784, 10])
    print()
    print(f"Original Accuracy: {net.test_accuracy() * 100}%")
    net.train(training_data, train_labels)
    print(f"After Training Accuracy: {net.test_accuracy() * 100}%")
    print()


Original Accuracy: 9.66%
After Training Accuracy: 13.309999999999999%


Original Accuracy: 11.31%


KeyboardInterrupt: 